In [128]:
import pandas as pd
import scipy.stats as st
from factor_analyzer import FactorAnalyzer
from sklearn.preprocessing import StandardScaler
from scipy import stats

In [129]:
df = pd.read_csv('data without fraud .csv')
df

,user_id,funnel_stage,timestamp,profit,os,ad_channel,price
0,666743,interest,150000,0.0,Android,banner,0.50
1,512217,interest,150000,0.0,iOS,video,0.75
2,399556,interest,150000,0.0,iOS,search,1.50
3,230119,interest,150000,0.0,iOS,search,1.50
4,619571,consideration,150000,0.0,iOS,search,1.50
...,...,...,...,...,...,...,...
979800,501946,intent,224125,0.0,Android,search,1.50
979801,749577,purchase,224647,2136.0,iOS,search,1.50
979802,929180,intent,224682,0.0,iOS,search,1.50
979803,1034498,intent,225430,0.0,iOS,search,1.50


### 1. Разделяем данные на две группы: органический и рекламный трафик

In [130]:
df['traffic_type'] = df['ad_channel'].apply(lambda x: 'organic' if x == 'organic traffic' else 'paid')
df

,user_id,funnel_stage,timestamp,profit,os,ad_channel,price,traffic_type
0,666743,interest,150000,0.0,Android,banner,0.50,paid
1,512217,interest,150000,0.0,iOS,video,0.75,paid
2,399556,interest,150000,0.0,iOS,search,1.50,paid
3,230119,interest,150000,0.0,iOS,search,1.50,paid
4,619571,consideration,150000,0.0,iOS,search,1.50,paid
...,...,...,...,...,...,...,...,...
979800,501946,intent,224125,0.0,Android,search,1.50,paid
979801,749577,purchase,224647,2136.0,iOS,search,1.50,paid
979802,929180,intent,224682,0.0,iOS,search,1.50,paid
979803,1034498,intent,225430,0.0,iOS,search,1.50,paid


In [131]:
order = ['interest', 'consideration', 'intent', 'purchase']
df['funnel_stage'] = pd.Categorical(df['funnel_stage'], categories=order, ordered=True)

In [132]:
df_type_traffic = df.groupby(['traffic_type', 'funnel_stage'])['user_id'].nunique()
df_type_traffic

traffic_type  funnel_stage 
organic       interest          77982
              consideration     53618
              intent             9079
              purchase            540
paid          interest         448546
              consideration    271517
              intent            38021
              purchase           3890
Name: user_id, dtype: int64

In [133]:
df_type_traffic = df_type_traffic.reset_index().pivot(
    index='traffic_type', 
    columns='funnel_stage', 
    values='user_id'  
).reset_index()
df_type_traffic

funnel_stage,traffic_type,interest,consideration,intent,purchase
0,organic,77982,53618,9079,540
1,paid,448546,271517,38021,3890


### 2. Сравниваем доли пользователей на каждом этапе воронки между двумя группами

In [134]:
organic = df_type_traffic[df_type_traffic['traffic_type'] == 'organic'].iloc[0]
paid = df_type_traffic[df_type_traffic['traffic_type'] == 'paid'].iloc[0]

organic_share_cons = organic['consideration'] / organic['interest'] 
paid_share_cons = paid['consideration'] / paid['interest']

organic_share_intent = organic['intent'] / organic['interest'] 
paid_share_intent = paid['intent'] / paid['interest']

organic_share_purchase = organic['purchase'] / organic['interest'] 
paid_share_purchase = paid['purchase'] / paid['interest']


print (f"Доля consideration для органического трафика: {organic_share_cons:.2%} для платного: {paid_share_cons:.2%}")
print (f"Доля intent для органического трафика: {organic_share_intent:.2%} для платного: {paid_share_intent:.2%}")
print (f"Доля purchase для органического трафика: {organic_share_purchase:.2%} для платного: {paid_share_purchase:.2%}")

Доля consideration для органического трафика: 68.76% для платного: 60.53%
Доля intent для органического трафика: 11.64% для платного: 8.48%
Доля purchase для органического трафика: 0.69% для платного: 0.87%


##### Consideration

In [135]:
## Используем биноминальный критерий для проверки гипотезы о равенстве долей (две выборки)

# Общее количество наблюдений в каждой группе
n1 = organic['interest'] 
n2 = paid['interest']
# Количество на конкретном этапе 
m1 = organic['consideration']
m2 = paid['consideration']

T = (m1/n1 - m2/n2)/((m1+m2)/(n1+n2)*(1 - (m1+m2)/(n1+n2))*(1/n1 + 1/n2))**0.5
P = 2 * (1 - stats.norm.cdf(abs(T)))
print("Statistic: ",T, ", p-value: ", P)

Statistic:  43.6164880792695 , p-value:  0.0


In [136]:
## Z-тест
from statsmodels.stats.proportion import proportions_ztest

successes = [m1, m2]
n_obs = [n1, n2]

Z, p = proportions_ztest(count=successes, nobs=n_obs)
print("Statistic: ",Z, ", p-value: ", p)

Statistic:  43.6164880792695 , p-value:  0.0


##### intent

In [137]:
## Биноминальный критерий
# Общее количество наблюдений в каждой группе
n1 = organic['interest'] 
n2 = paid['interest']
# Количество на конкретном этапе 
m1 = organic['intent']
m2 = paid['intent']

T = (m1/n1 - m2/n2)/((m1+m2)/(n1+n2)*(1 - (m1+m2)/(n1+n2))*(1/n1 + 1/n2))**0.5
P = 2 * (1 - stats.norm.cdf(abs(T)))
print("Statistic: ",T, ", p-value: ", P)

Statistic:  28.59174522004725 , p-value:  0.0


In [138]:
## Z-тест
from statsmodels.stats.proportion import proportions_ztest

successes = [m1, m2]
n_obs = [n1, n2]

Z, p = proportions_ztest(count=successes, nobs=n_obs)
print("Statistic: ",Z, ", p-value: ", p)

Statistic:  28.59174522004725 , p-value:  8.509987158273286e-180


##### purchase

In [139]:
## Биноминальный критерий
# Общее количество наблюдений в каждой группе
n1 = organic['interest'] 
n2 = paid['interest']
# Количество на конкретном этапе 
m1 = organic['purchase']
m2 = paid['purchase']

T = (m1/n1 - m2/n2)/((m1+m2)/(n1+n2)*(1 - (m1+m2)/(n1+n2))*(1/n1 + 1/n2))**0.5
P = 2 * (1 - stats.norm.cdf(abs(T)))
print("Statistic: ",T, ", p-value: ", P)

Statistic:  -4.932000251993781 , p-value:  8.139181180144561e-07


In [140]:
## Z-тест
from statsmodels.stats.proportion import proportions_ztest

successes = [m1, m2]
n_obs = [n1, n2]

Z, p = proportions_ztest(count=successes, nobs=n_obs)
print("Statistic: ",Z, ", p-value: ", p)

Statistic:  -4.932000251993781 , p-value:  8.13918118108495e-07


На всех этапах воронки разница между долями органического и платного трафика статистически значима.

### 3. Посчитаем воронки для каждого канала коммуникации

In [147]:
df_channel = df.groupby(['ad_channel', 'funnel_stage'])['user_id'].nunique()


In [142]:
df_channel = df_channel.reset_index().pivot(
    index='ad_channel', 
    columns='funnel_stage', 
    values='user_id'  
).reset_index()
df_channel

funnel_stage,ad_channel,interest,consideration,intent,purchase
0,banner,90994,47422,7539,688
1,native,19200,12583,406,36
2,organic traffic,77982,53618,9079,540
3,other,3185,1821,230,44
4,partner network,65388,24688,4370,450
5,rich,8830,4655,1221,21
6,search,222550,134050,27187,3085
7,video,97008,60872,58,5


In [146]:
df_conversions = df_channel[['ad_channel']].copy()
df_conversions['consideration'] = (df_channel['consideration'] / df_channel['interest'])*100
df_conversions['intent'] = (df_channel['intent'] / df_channel['consideration'])*100
df_conversions['purchase'] = (df_channel['purchase'] / df_channel['intent'])*100
df_conversions['total_conversion'] = (df_channel['purchase'] / df_channel['interest'])*100
df_conversions

funnel_stage,ad_channel,consideration,intent,purchase,total_conversion
0,banner,52.115524,15.897685,9.125879,0.756094
1,native,65.536458,3.226576,8.866995,0.187500
2,organic traffic,68.756893,16.932746,5.947792,0.692467
3,other,57.174254,12.630423,19.130435,1.381476
4,partner network,37.756163,17.700907,10.297483,0.688200
5,rich,52.718007,26.229860,1.719902,0.237826
6,search,60.233655,20.281238,11.347335,1.386205
7,video,62.749464,0.095282,8.620690,0.005154


### На этапе considerstion лучше всего показывет себя organic traffic, хуже всех partner network
### На этапе intent лучше всего показывет себя rich, хуже всех video
### На этапе purchase лучше всего показывет себя search, хуже всех rich

### 4. Результаты

#### Опираясь на общую конверсию в покупку наихудший канал video (у него самая низкая конверсия 0,005%), чуть выше конверсия у native и banner.
#### Очень мало пользователей доходят до этапа intent в каналах video (0,1%) и native (3,2%). На этом этапе хороший результат у rich (26,2%), но стоит обратить внимание, что у этого канала наименьшее количество привлеченных пользователей и низкая конверсия в покупку.
#### У partner network низкий переход с interest в consideration (37,7% в то время как у остальных более 50%)

#### Выключить стоит канал video очень низкая конверсия в intent и продаж практически нет, хотя охват у него неплохой, но ресурсы тратятся, а результатов практически нет.

#### Перераспределить показы стоит на каналлы:
#### - search - самый крупный канал с высокой конверсией в покупку. Увеличение показов скорее всего увеличат продажи;
#### - banner - тоже хорошие охваты и неплохая конверсия в покупку, на него тоже стоит обратить внимание;
#### - partner network — нужно доработать переход с interest в consideration, а так в целом у него неплохая конверсия;
#### - native - стоит поработать над переходом с consideration в intent;
#### - organic traffic - хороший источник, у него высокая конверсия на первых этапах, но снижается при переходе в покупку
      